In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

Matplotlib is building the font cache; this may take a moment.
/Users/joezhou/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:


def get_stock_data(ticker, years=5):
    """
    Fetch stock data for the given ticker for the specified number of years
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years*365)
    
    # Fetch historical data
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    # Fetch key financial metrics
    stock = yf.Ticker(ticker)
    
    # Get EPS
    try:
        eps = stock.info.get('trailingEPS', None)
        if eps is None:
            print("EPS data not available. Using default value.")
            eps = 0
    except:
        print("Error fetching EPS. Using default value.")
        eps = 0
    
    # Get growth rate (estimated from 5-year growth rate if available)
    try:
        growth_rate = stock.info.get('fiveYearAvgDividendYield', None)
        if growth_rate is None:
            print("Growth rate data not available. Using default value.")
            growth_rate = 0.05  # 5% default
        else:
            growth_rate = growth_rate / 100  # Convert percentage to decimal
    except:
        print("Error fetching growth rate. Using default value.")
        growth_rate = 0.05  # 5% default
    
    return stock_data, eps, growth_rate

def calculate_graham_value(eps, growth_rate, aaa_bond_yield=0.045):
    """
    Calculate Benjamin Graham's intrinsic value
    Formula: V = EPS × (8.5 + 2g) × 4.4 / Y
    
    Where:
    V = Intrinsic Value
    EPS = Earnings Per Share
    g = Growth Rate (decimal)
    Y = AAA Corporate Bond Yield (decimal)
    """
    if eps <= 0:
        return 0
    
    # Cap the growth rate at 0.25 (25%) as Graham suggested
    capped_growth_rate = min(growth_rate, 0.25)
    
    # Calculate the Graham value
    graham_value = eps * (8.5 + 2 * capped_growth_rate * 100) * 4.4 / (aaa_bond_yield * 100)
    
    return graham_value

def analyze_stock():
    """
    Main function to analyze a stock based on user input
    """
    # Get user input
    ticker = input("Enter stock ticker symbol (e.g., AAPL): ").strip().upper()
    aaa_bond_yield = float(input("Enter current AAA corporate bond yield (default 4.5%): ") or 4.5)
    
    # Fetch data
    print(f"\nFetching data for {ticker}...")
    stock_data, eps, growth_rate = get_stock_data(ticker)
    
    if stock_data.empty:
        print(f"No data available for ticker {ticker}")
        return
    
    # Calculate Graham value
    graham_value = calculate_graham_value(eps, growth_rate, aaa_bond_yield / 100)
    
    # Print results
    print("\n" + "="*50)
    print(f"Analysis for {ticker}")
    print("="*50)
    print(f"Current EPS: ${eps:.2f}")
    print(f"Estimated Growth Rate: {growth_rate*100:.2f}%")
    print(f"AAA Corporate Bond Yield: {aaa_bond_yield:.2f}%")
    print(f"Benjamin Graham Intrinsic Value: ${graham_value:.2f}")
    
    # Get current price
    current_price = stock_data['Close'].iloc[-1]
    print(f"Current Price: ${current_price:.2f}")
    
    # Calculate overvalued/undervalued percentage
    valuation_diff = ((current_price - graham_value) / graham_value) * 100
    
    if valuation_diff > 0:
        print(f"Stock is OVERVALUED by {abs(valuation_diff):.2f}%")
    else:
        print(f"Stock is UNDERVALUED by {abs(valuation_diff):.2f}%")
    
    # Plot the historical prices vs Graham value
    plt.figure(figsize=(12, 6))
    plt.plot(stock_data.index, stock_data['Close'], label='Historical Price')
    plt.axhline(y=graham_value, color='r', linestyle='-', label='Graham Value')
    plt.title(f"{ticker} Historical Price vs Graham Value")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Historical valuation analysis
    print("\n" + "="*50)
    print(f"Historical Valuation Analysis for {ticker}")
    print("="*50)
    
    # Calculate periods of over/undervaluation
    stock_data['Graham_Value'] = graham_value
    stock_data['Valuation_Diff'] = ((stock_data['Close'] - stock_data['Graham_Value']) / stock_data['Graham_Value']) * 100
    
    # Calculate percentage of time the stock was overvalued/undervalued
    overvalued_count = len(stock_data[stock_data['Valuation_Diff'] > 0])
    total_periods = len(stock_data)
    overvalued_percentage = (overvalued_count / total_periods) * 100
    undervalued_percentage = 100 - overvalued_percentage
    
    print(f"Over the past {len(stock_data) // 252} years:")
    print(f"Stock was overvalued {overvalued_percentage:.2f}% of the time")
    print(f"Stock was undervalued {undervalued_percentage:.2f}% of the time")
    
    # Find periods of maximum over/undervaluation
    max_overvalued = stock_data['Valuation_Diff'].max()
    max_overvalued_date = stock_data['Valuation_Diff'].idxmax()
    
    max_undervalued = stock_data['Valuation_Diff'].min()
    max_undervalued_date = stock_data['Valuation_Diff'].idxmin()
    
    print(f"\nMaximum overvaluation: {max_overvalued:.2f}% on {max_overvalued_date.date()}")
    print(f"Maximum undervaluation: {max_undervalued:.2f}% on {max_undervalued_date.date()}")
    
    # Additional valuation metrics
    print("\n" + "="*50)
    print(f"Additional Valuation Metrics for {ticker}")
    print("="*50)
    
    try:
        stock = yf.Ticker(ticker)
        pe_ratio = stock.info.get('trailingPE', 'N/A')
        pb_ratio = stock.info.get('priceToBook', 'N/A')
        dividend_yield = stock.info.get('dividendYield', 'N/A')
        if dividend_yield != 'N/A':
            dividend_yield = dividend_yield * 100  # Convert to percentage
            
        print(f"P/E Ratio: {pe_ratio if pe_ratio == 'N/A' else f'{pe_ratio:.2f}'}")
        print(f"P/B Ratio: {pb_ratio if pb_ratio == 'N/A' else f'{pb_ratio:.2f}'}")
        print(f"Dividend Yield: {dividend_yield if dividend_yield == 'N/A' else f'{dividend_yield:.2f}%'}")
    except:
        print("Unable to fetch additional metrics.")

if __name__ == "__main__":
    analyze_stock()


Fetching data for MSFT...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Error fetching EPS. Using default value.
Error fetching growth rate. Using default value.
No data available for ticker MSFT
